### Simulación de un día

In [3]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap
import matplotlib.cm as cm
import seaborn as sns
import os
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
import datetime
from sklearn.cluster import KMeans
from geopy.distance import geodesic 
import warnings
from collections import defaultdict
import requests

#### 40 estaciones

In [33]:
df = pd.read_csv("df_with_clusters.csv")
df['tsD'] = pd.to_datetime(df['tsD'], format='%d/%m/%Y %H:%M:%S', dayfirst=True)
df['tsO'] = pd.to_datetime(df['tsO'], format='%Y-%m-%d %H:%M:%S', dayfirst=False)

# Filtrar los datos para usar solo los del día 26 de febrero de 2021 
df = df[(df['tsO'].dt.date == pd.to_datetime('2021-02-26').date())]
# Dividir el DataFrame en dos y hacer copias para evitar el SettingWithCopyWarning
df_tsO_clusterO = df[['tsO', 'clusterO']].copy()
df_tsD_clusterD = df[['tsD', 'clusterD']].copy()

# Añadir columna 'tipo' para indicar si es un cluster de origen o destino
df_tsO_clusterO['tipo'] = 'origen'
df_tsD_clusterD['tipo'] = 'destino'

# Renombrar columnas para que tengan un nombre común ('ts' y 'cluster')
df_tsO_clusterO.rename(columns={'tsO': 'ts', 'clusterO': 'nodo'}, inplace=True)
df_tsD_clusterD.rename(columns={'tsD': 'ts', 'clusterD': 'nodo'}, inplace=True)

# Concatenar ambos DataFrames
df_combinado = pd.concat([df_tsO_clusterO, df_tsD_clusterD], ignore_index=True)

# Ordenar por la columna temporal 'ts'
df_combinado.sort_values(by='ts', inplace=True)

# Restablecer el índice
df_combinado.reset_index(drop=True, inplace=True)
# Verificar el resultado
print(df_combinado.head()), len(df), len(df_combinado)


                   ts  nodo     tipo
0 2021-02-26 00:15:23     2   origen
1 2021-02-26 00:36:54    23   origen
2 2021-02-26 00:38:43    23  destino
3 2021-02-26 00:55:44    28   origen
4 2021-02-26 00:59:43     2  destino


(None, 1110, 2220)

In [35]:
# Inicializar las estaciones
num_estaciones = 40
estaciones = {
    i: {
        "patinetes_iniciales":1,
        "patinetes_disponibles": 1,  # Inicialmente un patinete
        "slots": 1  # Inicialmente un slot
    }
    for i in range(num_estaciones)
}

# Simulación de eventos
for _, evento in df_combinado.iterrows():
    nodo = evento['nodo']
    tipo = evento['tipo']

    if tipo == 'origen':
        # Si es una salida, disminuir el número de patinetes disponibles
        if estaciones[nodo]['patinetes_disponibles'] > 0:
            estaciones[nodo]['patinetes_disponibles'] -= 1
        else:
            # Si no hay patinetes disponibles, aumentar el número de patinetes iniciales y slots (si fuese necesario)
            estaciones[nodo]['patinetes_iniciales'] += 1
            estaciones[nodo]['patinetes_disponibles'] = 0 #Se mantiene en 0 -> el patinete inicial adicional es el que se usa para al viaje
            if estaciones[nodo]['patinetes_iniciales'] > estaciones[nodo]['slots']:
                estaciones[nodo]['slots'] += 1

    elif tipo == 'destino':
        # Si es una llegada, aumentar el número de patinetes disponibles
        if estaciones[nodo]['patinetes_disponibles'] < estaciones[nodo]['slots']:
            estaciones[nodo]['patinetes_disponibles'] += 1
        else:
            # Si todos los slots están ocupados, aumentar el número de slots 
            estaciones[nodo]['slots'] += 1
            estaciones[nodo]['patinetes_disponibles'] += 1
            
print("Simulación terminada")
# Guardar los resultados en un archivo CSV
resultados = []
for nodo, datos in estaciones.items():
    resultados.append({
        'estacion': nodo,
        'patinetes_iniciales': datos['patinetes_iniciales'],
        'slots': datos['slots'],
        'patinetes_finales': datos['patinetes_disponibles'] 
    })

resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv('dimensiones.csv', index=False)
print("Resultados guardados en dimensiones.csv")
print(f"Total de patinetes usados: {resultados_df['patinetes_iniciales'].sum()}")

Simulación terminada
Resultados guardados en dimensiones.csv
Total de patinetes usados: 168


#### 79 estaciones

In [38]:
df = pd.read_csv("df_with_clusters2.csv")
df['tsD'] = pd.to_datetime(df['tsD'], format='%d/%m/%Y %H:%M:%S', dayfirst=True)
df['tsO'] = pd.to_datetime(df['tsO'], format='%Y-%m-%d %H:%M:%S', dayfirst=False)

# Filtrar los datos para usar solo los del día 26 de febrero de 2021 
df = df[(df['tsO'].dt.date == pd.to_datetime('2021-02-26').date())]
# Dividir el DataFrame en dos y hacer copias para evitar el SettingWithCopyWarning
df_tsO_clusterO = df[['tsO', 'clusterO']].copy()
df_tsD_clusterD = df[['tsD', 'clusterD']].copy()

# Añadir columna 'tipo' para indicar si es un cluster de origen o destino
df_tsO_clusterO['tipo'] = 'origen'
df_tsD_clusterD['tipo'] = 'destino'

# Renombrar columnas para que tengan un nombre común ('ts' y 'cluster')
df_tsO_clusterO.rename(columns={'tsO': 'ts', 'clusterO': 'nodo'}, inplace=True)
df_tsD_clusterD.rename(columns={'tsD': 'ts', 'clusterD': 'nodo'}, inplace=True)

# Concatenar ambos DataFrames
df_combinado = pd.concat([df_tsO_clusterO, df_tsD_clusterD], ignore_index=True)

# Ordenar por la columna temporal 'ts'
df_combinado.sort_values(by='ts', inplace=True)

# Restablecer el índice
df_combinado.reset_index(drop=True, inplace=True)
# Verificar el resultado
print(df_combinado.head()), len(df), len(df_combinado)


                   ts  nodo     tipo
0 2021-02-26 00:15:23    20   origen
1 2021-02-26 00:36:54     9   origen
2 2021-02-26 00:38:43     9  destino
3 2021-02-26 00:55:44    56   origen
4 2021-02-26 00:59:43    20  destino


(None, 1108, 2216)

In [40]:
# Inicializar las estaciones
num_estaciones = 79
estaciones = {
    i: {
        "patinetes_iniciales":1,
        "patinetes_disponibles": 1,  # Inicialmente un patinete
        "slots": 1  # Inicialmente un slot
    }
    for i in range(num_estaciones)
}

# Simulación de eventos
for _, evento in df_combinado.iterrows():
    nodo = evento['nodo']
    tipo = evento['tipo']

    if tipo == 'origen':
        # Si es una salida, disminuir el número de patinetes disponibles
        if estaciones[nodo]['patinetes_disponibles'] > 0:
            estaciones[nodo]['patinetes_disponibles'] -= 1
        else:
            # Si no hay patinetes disponibles, aumentar el número de patinetes iniciales y slots (si fuese necesario)
            estaciones[nodo]['patinetes_iniciales'] += 1
            estaciones[nodo]['patinetes_disponibles'] = 0 #Se mantiene en 0 -> el patinete inicial adicional es el que se usa para al viaje
            if estaciones[nodo]['patinetes_iniciales'] > estaciones[nodo]['slots']:
                estaciones[nodo]['slots'] += 1

    elif tipo == 'destino':
        # Si es una llegada, aumentar el número de patinetes disponibles
        if estaciones[nodo]['patinetes_disponibles'] < estaciones[nodo]['slots']:
            estaciones[nodo]['patinetes_disponibles'] += 1
        else:
            # Si todos los slots están ocupados, aumentar el número de slots 
            estaciones[nodo]['slots'] += 1
            estaciones[nodo]['patinetes_disponibles'] += 1
            
print("Simulación terminada")
# Guardar los resultados en un archivo CSV
resultados = []
for nodo, datos in estaciones.items():
    resultados.append({
        'estacion': nodo,
        'patinetes_iniciales': datos['patinetes_iniciales'],
        'slots': datos['slots'],
        'patinetes_finales': datos['patinetes_disponibles'] 
    })

resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv('dimensiones2.csv', index=False)
print("Resultados guardados en dimensiones2.csv")
print(f"Total de patinetes usados: {resultados_df['patinetes_iniciales'].sum()}")

Simulación terminada
Resultados guardados en dimensiones2.csv
Total de patinetes usados: 243


### Simulación semanal

#### 40 estaciones


In [43]:
# Cargar datos desde el archivo CSV
df = pd.read_csv("df_with_clusters.csv")
df['tsD'] = pd.to_datetime(df['tsD'], format='%d/%m/%Y %H:%M:%S', dayfirst=True)
df['tsO'] = pd.to_datetime(df['tsO'], format='%Y-%m-%d %H:%M:%S', dayfirst=False)

# Filtrar los datos para usar solo los eventos entre el 1 de febrero y el 7 de febrero de 2021
start_date = pd.to_datetime('2021-02-01')
end_date = pd.to_datetime('2021-02-07')

df = df[(df['tsO'].dt.date >= start_date.date()) & (df['tsO'].dt.date <= end_date.date())]

# Dividir el DataFrame en dos y hacer copias para evitar el SettingWithCopyWarning
df_tsO_clusterO = df[['tsO', 'clusterO']].copy()
df_tsD_clusterD = df[['tsD', 'clusterD']].copy()

# Añadir columna 'tipo' para indicar si es un cluster de origen o destino
df_tsO_clusterO['tipo'] = 'origen'
df_tsD_clusterD['tipo'] = 'destino'

# Renombrar columnas para que tengan un nombre común ('ts' y 'cluster')
df_tsO_clusterO.rename(columns={'tsO': 'ts', 'clusterO': 'nodo'}, inplace=True)
df_tsD_clusterD.rename(columns={'tsD': 'ts', 'clusterD': 'nodo'}, inplace=True)

# Concatenar ambos DataFrames
df_combinado = pd.concat([df_tsO_clusterO, df_tsD_clusterD], ignore_index=True)

# Ordenar por la columna temporal 'ts'
df_combinado.sort_values(by='ts', inplace=True)

# Restablecer el índice
df_combinado.reset_index(drop=True, inplace=True)
# Verificar el resultado
print(df_combinado.head()), len(df), len(df_combinado)

                   ts  nodo     tipo
0 2021-02-01 00:21:03     8   origen
1 2021-02-01 00:22:03     0  destino
2 2021-02-01 03:49:03    18   origen
3 2021-02-01 03:50:33    18  destino
4 2021-02-01 05:17:34    21   origen


(None, 5458, 10916)

In [45]:
# Inicializar las estaciones
num_estaciones = 40
estaciones = {
    i: {
        "patinetes_iniciales":1,
        "patinetes_disponibles": 1,  # Inicialmente un patinete
        "slots": 1  # Inicialmente un slot
    }
    for i in range(num_estaciones)
}

# Simulación de eventos
for _, evento in df_combinado.iterrows():
    nodo = evento['nodo']
    tipo = evento['tipo']

    if tipo == 'origen':
        # Si es una salida, disminuir el número de patinetes disponibles
        if estaciones[nodo]['patinetes_disponibles'] > 0:
            estaciones[nodo]['patinetes_disponibles'] -= 1
        else:
            # Si no hay patinetes disponibles, aumentar el número de patinetes iniciales y slots (si fuese necesario)
            estaciones[nodo]['patinetes_iniciales'] += 1
            estaciones[nodo]['patinetes_disponibles'] = 0 #Se mantiene en 0 -> el patinete inicial adicional es el que se usa para al viaje
            if estaciones[nodo]['patinetes_iniciales'] > estaciones[nodo]['slots']:
                estaciones[nodo]['slots'] += 1

    elif tipo == 'destino':
        # Si es una llegada, aumentar el número de patinetes disponibles
        if estaciones[nodo]['patinetes_disponibles'] < estaciones[nodo]['slots']:
            estaciones[nodo]['patinetes_disponibles'] += 1
        else:
            # Si todos los slots están ocupados, aumentar el número de slots 
            estaciones[nodo]['slots'] += 1
            estaciones[nodo]['patinetes_disponibles'] += 1
            
print("Simulación terminada")
# Guardar los resultados en un archivo CSV
resultados = []
for nodo, datos in estaciones.items():
    resultados.append({
        'estacion': nodo,
        'patinetes_iniciales': datos['patinetes_iniciales'],
        'slots': datos['slots'],
        'patinetes_finales': datos['patinetes_disponibles'] 
    })

resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv('dimensiones3.csv', index=False)
print("Resultados guardados en dimensiones3.csv")
print(f"Total de patinetes usados: {resultados_df['patinetes_iniciales'].sum()}")

Simulación terminada
Resultados guardados en dimensiones3.csv
Total de patinetes usados: 338


#### 79 estaciones

In [59]:
# Cargar datos desde el archivo CSV
df = pd.read_csv("df_with_clusters2.csv")
df['tsD'] = pd.to_datetime(df['tsD'], format='%d/%m/%Y %H:%M:%S', dayfirst=True)
df['tsO'] = pd.to_datetime(df['tsO'], format='%Y-%m-%d %H:%M:%S', dayfirst=False)

# Filtrar los datos para usar solo los eventos entre el 1 de febrero y el 7 de febrero de 2021
start_date = pd.to_datetime('2021-02-01')
end_date = pd.to_datetime('2021-02-07')

df = df[(df['tsO'].dt.date >= start_date.date()) & (df['tsO'].dt.date <= end_date.date())]

# Dividir el DataFrame en dos y hacer copias para evitar el SettingWithCopyWarning
df_tsO_clusterO = df[['tsO', 'clusterO']].copy()
df_tsD_clusterD = df[['tsD', 'clusterD']].copy()

# Añadir columna 'tipo' para indicar si es un cluster de origen o destino
df_tsO_clusterO['tipo'] = 'origen'
df_tsD_clusterD['tipo'] = 'destino'

# Renombrar columnas para que tengan un nombre común ('ts' y 'cluster')
df_tsO_clusterO.rename(columns={'tsO': 'ts', 'clusterO': 'nodo'}, inplace=True)
df_tsD_clusterD.rename(columns={'tsD': 'ts', 'clusterD': 'nodo'}, inplace=True)

# Concatenar ambos DataFrames
df_combinado = pd.concat([df_tsO_clusterO, df_tsD_clusterD], ignore_index=True)

# Ordenar por la columna temporal 'ts'
df_combinado.sort_values(by='ts', inplace=True)

# Restablecer el índice
df_combinado.reset_index(drop=True, inplace=True)
# Verificar el resultado
print(df_combinado.head()), len(df), len(df_combinado)

                   ts  nodo     tipo
0 2021-02-01 00:21:03    33   origen
1 2021-02-01 00:22:03    14  destino
2 2021-02-01 03:49:03    43   origen
3 2021-02-01 03:50:33    43  destino
4 2021-02-01 05:17:34    45   origen


(None, 5439, 10878)

In [61]:
# Inicializar las estaciones
num_estaciones = 79
estaciones = {
    i: {
        "patinetes_iniciales":1,
        "patinetes_disponibles": 1,  # Inicialmente un patinete
        "slots": 1  # Inicialmente un slot
    }
    for i in range(num_estaciones)
}

# Simulación de eventos
for _, evento in df_combinado.iterrows():
    nodo = evento['nodo']
    tipo = evento['tipo']

    if tipo == 'origen':
        # Si es una salida, disminuir el número de patinetes disponibles
        if estaciones[nodo]['patinetes_disponibles'] > 0:
            estaciones[nodo]['patinetes_disponibles'] -= 1
        else:
            # Si no hay patinetes disponibles, aumentar el número de patinetes iniciales y slots (si fuese necesario)
            estaciones[nodo]['patinetes_iniciales'] += 1
            estaciones[nodo]['patinetes_disponibles'] = 0 #Se mantiene en 0 -> el patinete inicial adicional es el que se usa para al viaje
            if estaciones[nodo]['patinetes_iniciales'] > estaciones[nodo]['slots']:
                estaciones[nodo]['slots'] += 1

    elif tipo == 'destino':
        # Si es una llegada, aumentar el número de patinetes disponibles
        if estaciones[nodo]['patinetes_disponibles'] < estaciones[nodo]['slots']:
            estaciones[nodo]['patinetes_disponibles'] += 1
        else:
            # Si todos los slots están ocupados, aumentar el número de slots 
            estaciones[nodo]['slots'] += 1
            estaciones[nodo]['patinetes_disponibles'] += 1
            
print("Simulación terminada")
# Guardar los resultados en un archivo CSV
resultados = []
for nodo, datos in estaciones.items():
    resultados.append({
        'estacion': nodo,
        'patinetes_iniciales': datos['patinetes_iniciales'],
        'slots': datos['slots'],
        'patinetes_finales': datos['patinetes_disponibles'] 
    })

resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv('dimensiones4.csv', index=False)
print("Resultados guardados en dimensiones4.csv")
print(f"Total de patinetes usados: {resultados_df['patinetes_iniciales'].sum()}")

Simulación terminada
Resultados guardados en dimensiones4.csv
Total de patinetes usados: 547


### Simulación de un mes entero

#### 40 estaciones

In [40]:
# Leer el archivo CSV
df = pd.read_csv("df_with_clusters.csv")
df['tsD'] = pd.to_datetime(df['tsD'], format='%d/%m/%Y %H:%M:%S', dayfirst=True)
df['tsO'] = pd.to_datetime(df['tsO'], format='%Y-%m-%d %H:%M:%S', dayfirst=False)

# Crear DataFrame combinado
df_tsO_clusterO = df[['tsO', 'clusterO']].copy()
df_tsD_clusterD = df[['tsD', 'clusterD']].copy()

df_tsO_clusterO['tipo'] = 'origen'
df_tsD_clusterD['tipo'] = 'destino'

df_tsO_clusterO.rename(columns={'tsO': 'ts', 'clusterO': 'nodo'}, inplace=True)
df_tsD_clusterD.rename(columns={'tsD': 'ts', 'clusterD': 'nodo'}, inplace=True)

df_combinado = pd.concat([df_tsO_clusterO, df_tsD_clusterD], ignore_index=True)
df_combinado.sort_values(by='ts', inplace=True)
df_combinado.reset_index(drop=True, inplace=True)
print(df_combinado.head()), len(df), len(df_combinado)

                   ts  nodo     tipo
0 2021-02-01 00:21:03     8   origen
1 2021-02-01 00:22:03     0  destino
2 2021-02-01 03:49:03    18   origen
3 2021-02-01 03:50:33    18  destino
4 2021-02-01 05:17:34    21   origen


(None, 23984, 47968)

In [42]:
# Inicializar las estaciones
from datetime import datetime as dt, timedelta
# Inicializar las estaciones
num_estaciones = 40
estaciones = {
    i: {
        "patinetes_iniciales":1,
        "patinetes_disponibles": 1,  # Inicialmente un patinete
        "slots": 1  # Inicialmente un slot
    }
    for i in range(num_estaciones)
}

# Filtrar datos por día y simular
inicio_mes = dt(2021, 2, 1)
fin_mes = dt(2021, 2, 28)

for dia in range((fin_mes - inicio_mes).days + 1):
    fecha_actual = inicio_mes + timedelta(days=dia)
    siguiente_dia = fecha_actual + timedelta(days=1)

    # Filtrar datos del día actual
    df_dia = df_combinado[(df_combinado['ts'] >= fecha_actual) & (df_combinado['ts'] < siguiente_dia)]

    # Simular eventos
    for _, evento in df_dia.iterrows():
        nodo = evento['nodo']
        tipo = evento['tipo']

        if tipo == 'origen':
            if estaciones[nodo]['patinetes_disponibles'] > 0:
                estaciones[nodo]['patinetes_disponibles'] -= 1
            else:
                estaciones[nodo]['patinetes_iniciales'] += 1
                estaciones[nodo]['patinetes_disponibles'] = 0
                if estaciones[nodo]['patinetes_iniciales'] > estaciones[nodo]['slots']:
                    estaciones[nodo]['slots'] += 1

        elif tipo == 'destino':
            if estaciones[nodo]['patinetes_disponibles'] < estaciones[nodo]['slots']:
                estaciones[nodo]['patinetes_disponibles'] += 1
            else:
                estaciones[nodo]['slots'] += 1
                estaciones[nodo]['patinetes_disponibles'] += 1

    # Actualizar las estaciones globales al final del día
    for nodo, datos in estaciones.items():
        estaciones[nodo]['patinetes_disponibles'] = estaciones[nodo]['patinetes_iniciales']
        

print("Simulación terminada")

# Guardar los resultados en un archivo CSV
resultados = []
for nodo, datos in estaciones.items():
    resultados.append({
        'estacion': nodo,
        'patinetes_iniciales': datos['patinetes_iniciales'],
        'slots': datos['slots']
    })

resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv('dimensiones_mes1.csv', index=False)
print("Resultados guardados en dimensiones_mes1.csv")
print(f"Total de patinetes usados: {resultados_df['patinetes_iniciales'].sum()}")

Simulación terminada
Resultados guardados en dimensiones_mes1.csv
Total de patinetes usados: 364


#### 79 estaciones

In [44]:
# Leer el archivo CSV
df = pd.read_csv("df_with_clusters2.csv")
df['tsD'] = pd.to_datetime(df['tsD'], format='%d/%m/%Y %H:%M:%S', dayfirst=True)
df['tsO'] = pd.to_datetime(df['tsO'], format='%Y-%m-%d %H:%M:%S', dayfirst=False)

# Crear DataFrame combinado
df_tsO_clusterO = df[['tsO', 'clusterO']].copy()
df_tsD_clusterD = df[['tsD', 'clusterD']].copy()

df_tsO_clusterO['tipo'] = 'origen'
df_tsD_clusterD['tipo'] = 'destino'

df_tsO_clusterO.rename(columns={'tsO': 'ts', 'clusterO': 'nodo'}, inplace=True)
df_tsD_clusterD.rename(columns={'tsD': 'ts', 'clusterD': 'nodo'}, inplace=True)

df_combinado = pd.concat([df_tsO_clusterO, df_tsD_clusterD], ignore_index=True)
df_combinado.sort_values(by='ts', inplace=True)
df_combinado.reset_index(drop=True, inplace=True)
print(df_combinado.head()), len(df), len(df_combinado)

                   ts  nodo     tipo
0 2021-02-01 00:21:03    33   origen
1 2021-02-01 00:22:03    14  destino
2 2021-02-01 03:49:03    43   origen
3 2021-02-01 03:50:33    43  destino
4 2021-02-01 05:17:34    45   origen


(None, 23868, 47736)

In [45]:
# Inicializar las estaciones
from datetime import datetime as dt, timedelta
# Inicializar las estaciones
num_estaciones = 79
estaciones = {
    i: {
        "patinetes_iniciales":1,
        "patinetes_disponibles": 1,  # Inicialmente un patinete
        "slots": 1  # Inicialmente un slot
    }
    for i in range(num_estaciones)
}

# Filtrar datos por día y simular
inicio_mes = dt(2021, 2, 1)
fin_mes = dt(2021, 2, 28)

for dia in range((fin_mes - inicio_mes).days + 1):
    fecha_actual = inicio_mes + timedelta(days=dia)
    siguiente_dia = fecha_actual + timedelta(days=1)

    # Filtrar datos del día actual
    df_dia = df_combinado[(df_combinado['ts'] >= fecha_actual) & (df_combinado['ts'] < siguiente_dia)]

    # Simular eventos
    for _, evento in df_dia.iterrows():
        nodo = evento['nodo']
        tipo = evento['tipo']

        if tipo == 'origen':
            if estaciones[nodo]['patinetes_disponibles'] > 0:
                estaciones[nodo]['patinetes_disponibles'] -= 1
            else:
                estaciones[nodo]['patinetes_iniciales'] += 1
                estaciones[nodo]['patinetes_disponibles'] = 0
                if estaciones[nodo]['patinetes_iniciales'] > estaciones[nodo]['slots']:
                    estaciones[nodo]['slots'] += 1

        elif tipo == 'destino':
            if estaciones[nodo]['patinetes_disponibles'] < estaciones[nodo]['slots']:
                estaciones[nodo]['patinetes_disponibles'] += 1
            else:
                estaciones[nodo]['slots'] += 1
                estaciones[nodo]['patinetes_disponibles'] += 1

    # Actualizar las estaciones globales al final del día
    for nodo, datos in estaciones.items():
        estaciones[nodo]['patinetes_disponibles'] = estaciones[nodo]['patinetes_iniciales']
        

print("Simulación terminada")

# Guardar los resultados en un archivo CSV
resultados = []
for nodo, datos in estaciones.items():
    resultados.append({
        'estacion': nodo,
        'patinetes_iniciales': datos['patinetes_iniciales'],
        'slots': datos['slots']
    })

resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv('dimensiones_mes2.csv', index=False)
print("Resultados guardados en dimensiones_mes2.csv")
print(f"Total de patinetes usados: {resultados_df['patinetes_iniciales'].sum()}")

Simulación terminada
Resultados guardados en dimensiones_mes2.csv
Total de patinetes usados: 581


### Simulación de un mes por semanas

#### 40 estaciones

In [48]:
# Leer el archivo CSV
df = pd.read_csv("df_with_clusters.csv")
df['tsD'] = pd.to_datetime(df['tsD'], format='%d/%m/%Y %H:%M:%S', dayfirst=True)
df['tsO'] = pd.to_datetime(df['tsO'], format='%Y-%m-%d %H:%M:%S', dayfirst=False)

# Crear DataFrame combinado
df_tsO_clusterO = df[['tsO', 'clusterO']].copy()
df_tsD_clusterD = df[['tsD', 'clusterD']].copy()

df_tsO_clusterO['tipo'] = 'origen'
df_tsD_clusterD['tipo'] = 'destino'

df_tsO_clusterO.rename(columns={'tsO': 'ts', 'clusterO': 'nodo'}, inplace=True)
df_tsD_clusterD.rename(columns={'tsD': 'ts', 'clusterD': 'nodo'}, inplace=True)

df_combinado = pd.concat([df_tsO_clusterO, df_tsD_clusterD], ignore_index=True)
df_combinado.sort_values(by='ts', inplace=True)
df_combinado.reset_index(drop=True, inplace=True)
print(df_combinado.head()), len(df), len(df_combinado)

                   ts  nodo     tipo
0 2021-02-01 00:21:03     8   origen
1 2021-02-01 00:22:03     0  destino
2 2021-02-01 03:49:03    18   origen
3 2021-02-01 03:50:33    18  destino
4 2021-02-01 05:17:34    21   origen


(None, 23984, 47968)

In [50]:
# Inicializar las estaciones
from datetime import datetime as dt, timedelta
# Inicializar las estaciones
num_estaciones = 40
estaciones = {
    i: {
        "patinetes_iniciales":1,
        "patinetes_disponibles": 1,  # Inicialmente un patinete
        "slots": 1  # Inicialmente un slot
    }
    for i in range(num_estaciones)
}

# Filtrar datos por día y simular
inicio_mes = dt(2021, 2, 1)
fin_mes = dt(2021, 2, 28)

for dia in range((fin_mes - inicio_mes).days + 1):
    fecha_actual = inicio_mes + timedelta(days=dia)
    siguiente_dia = fecha_actual + timedelta(days=1)

    # Filtrar datos del día actual
    df_dia = df_combinado[(df_combinado['ts'] >= fecha_actual) & (df_combinado['ts'] < siguiente_dia)]

    # Simular eventos
    for _, evento in df_dia.iterrows():
        nodo = evento['nodo']
        tipo = evento['tipo']

        if tipo == 'origen':
            if estaciones[nodo]['patinetes_disponibles'] > 0:
                estaciones[nodo]['patinetes_disponibles'] -= 1
            else:
                estaciones[nodo]['patinetes_iniciales'] += 1
                estaciones[nodo]['patinetes_disponibles'] = 0
                if estaciones[nodo]['patinetes_iniciales'] > estaciones[nodo]['slots']:
                    estaciones[nodo]['slots'] += 1

        elif tipo == 'destino':
            if estaciones[nodo]['patinetes_disponibles'] < estaciones[nodo]['slots']:
                estaciones[nodo]['patinetes_disponibles'] += 1
            else:
                estaciones[nodo]['slots'] += 1
                estaciones[nodo]['patinetes_disponibles'] += 1

    # Realizar la redistribución semanal al final del domingo
    if fecha_actual.weekday() == 0:  # Domingo
        for nodo, datos in estaciones.items():
            estaciones[nodo]['patinetes_disponibles'] = estaciones[nodo]['patinetes_iniciales']

print("Simulación terminada")

# Guardar los resultados en un archivo CSV
resultados = []
for nodo, datos in estaciones.items():
    resultados.append({
        'estacion': nodo,
        'patinetes_iniciales': datos['patinetes_iniciales'],
        'slots': datos['slots']
    })

resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv('dimensiones_semanas1.csv', index=False)
print("Resultados guardados en dimensiones_semanas1.csv")
print(f"Total de patinetes usados: {resultados_df['patinetes_iniciales'].sum()}")

Simulación terminada
Resultados guardados en dimensiones_semanas1.csv
Total de patinetes usados: 611


#### 79 estaciones

In [56]:
# Leer el archivo CSV
df = pd.read_csv("df_with_clusters2.csv")
df['tsD'] = pd.to_datetime(df['tsD'], format='%d/%m/%Y %H:%M:%S', dayfirst=True)
df['tsO'] = pd.to_datetime(df['tsO'], format='%Y-%m-%d %H:%M:%S', dayfirst=False)

# Crear DataFrame combinado
df_tsO_clusterO = df[['tsO', 'clusterO']].copy()
df_tsD_clusterD = df[['tsD', 'clusterD']].copy()

df_tsO_clusterO['tipo'] = 'origen'
df_tsD_clusterD['tipo'] = 'destino'

df_tsO_clusterO.rename(columns={'tsO': 'ts', 'clusterO': 'nodo'}, inplace=True)
df_tsD_clusterD.rename(columns={'tsD': 'ts', 'clusterD': 'nodo'}, inplace=True)

df_combinado = pd.concat([df_tsO_clusterO, df_tsD_clusterD], ignore_index=True)
df_combinado.sort_values(by='ts', inplace=True)
df_combinado.reset_index(drop=True, inplace=True)
print(df_combinado.head()), len(df), len(df_combinado)

                   ts  nodo     tipo
0 2021-02-01 00:21:03    33   origen
1 2021-02-01 00:22:03    14  destino
2 2021-02-01 03:49:03    43   origen
3 2021-02-01 03:50:33    43  destino
4 2021-02-01 05:17:34    45   origen


(None, 23868, 47736)

In [58]:
# Inicializar las estaciones
from datetime import datetime as dt, timedelta
# Inicializar las estaciones
num_estaciones = 79
estaciones = {
    i: {
        "patinetes_iniciales":1,
        "patinetes_disponibles": 1,  # Inicialmente un patinete
        "slots": 1  # Inicialmente un slot
    }
    for i in range(num_estaciones)
}

# Filtrar datos por día y simular
inicio_mes = dt(2021, 2, 1)
fin_mes = dt(2021, 2, 28)

for dia in range((fin_mes - inicio_mes).days + 1):
    fecha_actual = inicio_mes + timedelta(days=dia)
    siguiente_dia = fecha_actual + timedelta(days=1)

    # Filtrar datos del día actual
    df_dia = df_combinado[(df_combinado['ts'] >= fecha_actual) & (df_combinado['ts'] < siguiente_dia)]

    # Simular eventos
    for _, evento in df_dia.iterrows():
        nodo = evento['nodo']
        tipo = evento['tipo']

        if tipo == 'origen':
            if estaciones[nodo]['patinetes_disponibles'] > 0:
                estaciones[nodo]['patinetes_disponibles'] -= 1
            else:
                estaciones[nodo]['patinetes_iniciales'] += 1
                estaciones[nodo]['patinetes_disponibles'] = 0
                if estaciones[nodo]['patinetes_iniciales'] > estaciones[nodo]['slots']:
                    estaciones[nodo]['slots'] += 1

        elif tipo == 'destino':
            if estaciones[nodo]['patinetes_disponibles'] < estaciones[nodo]['slots']:
                estaciones[nodo]['patinetes_disponibles'] += 1
            else:
                estaciones[nodo]['slots'] += 1
                estaciones[nodo]['patinetes_disponibles'] += 1

    # Realizar la redistribución semanal al final del domingo
    if fecha_actual.weekday() == 0:  # Domingo
        for nodo, datos in estaciones.items():
            estaciones[nodo]['patinetes_disponibles'] = estaciones[nodo]['patinetes_iniciales']

print("Simulación terminada")

# Guardar los resultados en un archivo CSV
resultados = []
for nodo, datos in estaciones.items():
    resultados.append({
        'estacion': nodo,
        'patinetes_iniciales': datos['patinetes_iniciales'],
        'slots': datos['slots']
    })

resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv('dimensiones_semanas2.csv', index=False)
print("Resultados guardados en dimensiones_semanas2.csv")
print(f"Total de patinetes usados: {resultados_df['patinetes_iniciales'].sum()}")

Simulación terminada
Resultados guardados en dimensiones_semanas2.csv
Total de patinetes usados: 1028


#### Altitudes


In [6]:
df = pd.read_csv("cluster_metrics.csv")
# Supongamos que tu DataFrame se llama df y tiene las columnas 'Centroide_Lat' y 'Centroide_Lon'
# Redondear las coordenadas a 3 decimales
df['Lat_Rounded'] = df['Centroide_Lat'].round(3)
df['Lon_Rounded'] = df['Centroide_Lon'].round(3)

# Eliminar filas duplicadas basadas en las coordenadas redondeadas
unique_coords = df[['Lat_Rounded', 'Lon_Rounded']].drop_duplicates()

# Crear un diccionario para almacenar las altitudes
altitudes = {}

# Iterar sobre las coordenadas únicas y hacer solicitudes a la API
for _, row in unique_coords.iterrows():
    lat = row['Lat_Rounded']
    lon = row['Lon_Rounded']
    url = f'https://api.open-meteo.com/v1/elevation?latitude={lat}&longitude={lon}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        elevation = data.get('elevation', None)
        if elevation is not None:
            # Convertir la altitud a cadena y eliminar corchetes si existen
            elevation_str = str(elevation).replace('[', '').replace(']', '')
            altitudes[(lat, lon)] = elevation_str
        else:
            altitudes[(lat, lon)] = None
    else:
        altitudes[(lat, lon)] = None

# Crear una nueva columna en el DataFrame original para las altitudes
df['Altitud'] = df.apply(lambda x: altitudes.get((x['Lat_Rounded'], x['Lon_Rounded'])), axis=1)

# Eliminar las columnas temporales
df.drop(columns=['Lat_Rounded', 'Lon_Rounded'], inplace=True)
df.to_csv('cluster_metrics.csv', index=False)
df

,Cluster_ID,Centroide_Lat,Centroide_Lon,Distancia_Media_km,Num_Puntos,Pct_Origen,Pct_Destino,Altitud
0,0,41.879856,12.510375,0.447526,3037,50.444518,49.555482,44.0
1,1,41.909890,12.472796,0.415460,1764,50.850340,49.149660,10.0
2,2,41.855957,12.479274,0.393418,1123,50.578807,49.421193,14.0
3,3,41.914119,12.521666,0.438023,894,49.776286,50.223714,45.0
4,4,41.904979,12.511413,0.483289,432,47.916667,52.083333,62.0
5,5,41.888195,12.471115,0.349055,1869,49.491707,50.508293,30.0
6,6,41.892500,12.492634,0.384945,2040,50.833333,49.166667,50.0
7,7,41.881260,12.459078,0.344616,2040,49.558824,50.441176,72.0
8,8,41.885580,12.520644,0.377824,1593,47.080979,52.919021,54.0
9,9,41.918420,12.459865,0.386741,973,52.312436,47.687564,19.0


In [8]:
df = pd.read_csv("cluster_metrics2.csv")
# Supongamos que tu DataFrame se llama df y tiene las columnas 'Centroide_Lat' y 'Centroide_Lon'
# Redondear las coordenadas a 3 decimales
df['Lat_Rounded'] = df['Centroide_Lat'].round(3)
df['Lon_Rounded'] = df['Centroide_Lon'].round(3)

# Eliminar filas duplicadas basadas en las coordenadas redondeadas
unique_coords = df[['Lat_Rounded', 'Lon_Rounded']].drop_duplicates()

# Crear un diccionario para almacenar las altitudes
altitudes = {}

# Iterar sobre las coordenadas únicas y hacer solicitudes a la API
for _, row in unique_coords.iterrows():
    lat = row['Lat_Rounded']
    lon = row['Lon_Rounded']
    url = f'https://api.open-meteo.com/v1/elevation?latitude={lat}&longitude={lon}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        elevation = data.get('elevation', None)
        if elevation is not None:
            # Convertir la altitud a cadena y eliminar corchetes si existen
            elevation_str = str(elevation).replace('[', '').replace(']', '')
            altitudes[(lat, lon)] = elevation_str
        else:
            altitudes[(lat, lon)] = None
    else:
        altitudes[(lat, lon)] = None

# Crear una nueva columna en el DataFrame original para las altitudes
df['Altitud'] = df.apply(lambda x: altitudes.get((x['Lat_Rounded'], x['Lon_Rounded'])), axis=1)

# Eliminar las columnas temporales
df.drop(columns=['Lat_Rounded', 'Lon_Rounded'], inplace=True)
df.to_csv('cluster_metrics2.csv', index=False)
df

,Cluster_ID,Centroide_Lat,Centroide_Lon,Distancia_Media_km,Num_Puntos,Pct_Origen,Pct_Destino,Altitud
0,0,41.904979,12.511413,0.483289,432,47.916667,52.083333,62.0
1,1,41.925987,12.466829,0.447158,591,52.453469,47.546531,19.0
2,2,41.830730,12.472889,0.402806,280,49.642857,50.357143,31.0
3,3,41.822503,12.478084,0.451850,314,47.133758,52.866242,34.0
4,4,41.931340,12.519139,0.437415,580,47.931034,52.068966,42.0
...,...,...,...,...,...,...,...,...
74,74,41.844364,12.483699,0.234220,822,48.783455,51.216545,39.0
75,75,41.848672,12.486664,0.220004,475,56.210526,43.789474,35.0
76,76,41.846385,12.490081,0.215952,372,47.580645,52.419355,48.0
77,77,41.910376,12.447078,0.347202,503,52.882704,47.117296,41.0


## 40 VS 79

In [25]:
df = pd.read_csv("df_with_clusters.csv")
df_intra_cluster = df[df['clusterO'] == df['clusterD']]
df_intra_cluster['tt'].describe()

count    8883.000000
mean      500.931217
std       577.219765
min        60.000000
25%       160.000000
50%       280.000000
75%       589.500000
max      5011.000000
Name: tt, dtype: float64

In [52]:
# Calcular la cantidad de filas que se perderían
filas_perdidas = 0.147 * len(df) #Hay un 37% de viajes intra-cluster con 40 clúster y un 22.3% con 79 clústers

# Calcular el valor asociado a las filas perdidas: nª de filas * tiempo medio de esos trayectos * precio por segundo
valor_perdido = filas_perdidas * np.mean(df_intra_cluster['tt']) * (sum(df_intra_cluster['price']) / sum(df_intra_cluster['tt']))

# Imprimir los resultados con formato
print(f'Con 40 clústers se perderían {filas_perdidas:,.0f} filas.')
print(f'Se perderían {valor_perdido:,.2f} €')


Con 40 clústers se perderían 3,526 filas.
Se perderían 7,940.92 €
